In [1]:
# Import necessary libraries
from transformers import pipeline
import numpy as np
import asyncio
from metrics import AnswerSimilarity

ImportError: attempted relative import with no known parent package

In [1]:
import sys
sys.path.append("../")

from dotenv import load_dotenv
load_dotenv("../../.env")

True

In [3]:
# Create an instance of the class
similarity_scorer = AnswerSimilarity(threshold=0.5)

# Use the class with strings and return binary
ground_truth_string = "This is a test."
answer_string = "This is a test."
binary_score_string = similarity_scorer.score(ground_truth_string, answer_string, return_binary=True)
print(f"Binary Score (strings): {binary_score_string}")

# Use the class with embeddings and return score
ground_truth_embedding = np.random.rand(768)
answer_embedding = np.random.rand(768)
score_embedding = similarity_scorer.score(ground_truth_embedding, answer_embedding, return_binary=False)
print(f"Score (embeddings): {score_embedding}")

Binary Score (strings): 0
Score (embeddings): 0.749


In [5]:

metrics = ["faithfulness", "answer_correctness"]
question = ['When was the first super bowl?', 'Who won the most super bowls?']
answer = ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots']
context = [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
           ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']]
ground_truth = ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']

In [6]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, answer_relevancy, answer_similarity, context_entity_recall, context_precision, context_recall, context_relevancy

class MetricsEvaluator:
    def __init__(self):
        self.metrics_dict = {
            'faithfulness': faithfulness, 
            'answer_correctness': answer_correctness,
            'answer_relevancy': answer_relevancy,
            'answer_similarity': answer_similarity,
            'context_entity_recall': context_entity_recall,
            'context_precision': context_precision,
            'context_recall': context_recall,
            'context_relevancy': context_relevancy,                 
        }

    def score(self, metrics, question, answer, contexts, ground_truth):
        # Create a list of metrics modules based on the input list of strings
        metrics_modules = [self.metrics_dict[metric] for metric in metrics if metric in self.metrics_dict]

        data_samples = {
            'question': question,
            'answer': answer,
            'contexts': contexts,
            'ground_truth': ground_truth
        }

        dataset = Dataset.from_dict(data_samples)

        # Use the list of metrics modules in the evaluate function
        score = evaluate(dataset, metrics=metrics_modules)
        return score.to_pandas()

In [7]:

evaluator = MetricsEvaluator()
metrics = list(evaluator.metrics_dict.keys())
question = ['When was the first super bowl?', 'Who won the most super bowls?']
answer = ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots']
contexts = [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']]
ground_truth = ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']

scores = evaluator.score(metrics, question, answer, contexts, ground_truth)
print(scores)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARAL

                         question  \
0  When was the first super bowl?   
1   Who won the most super bowls?   

                                              answer  \
0       The first superbowl was held on Jan 15, 1967   
1  The most super bowls have been won by The New ...   

                                            contexts  \
0  [The First AFL–NFL World Championship Game was...   
1  [The Green Bay Packers...Green Bay, Wisconsin....   

                                        ground_truth  faithfulness  \
0   The first superbowl was held on January 15, 1967           1.0   
1  The New England Patriots have won the Super Bo...           NaN   

   answer_correctness  answer_relevancy  answer_similarity  \
0            0.749093          0.980723           0.996372   
1            0.731086          0.943010           0.924343   

   context_entity_recall  context_precision  context_recall  context_relevancy  
0                    0.5                1.0             1.0            